In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import *
from torch.utils.data import Dataset
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [19]:
class NN(nn.Module):
    def __init__(self, C_in, C_out, widths, Layers, act_func, w_initilization):
        super(NN, self).__init__()
        
        layers=[]
        self.w_initilization=w_initilization
        
        for i in range(Layers):
            if i == 0:
                 layers += [nn.Linear(C_in, widths)]
            elif i == Layers - 1:
                 layers += [nn.Linear(widths, C_out)]
            else:
                 layers += [nn.Linear(widths, widths)]
                    
            #try two activation functions:"tanh" and "RELU"
            if act == 'relu':
                layers += [nn.ReLU()]
            else:
                layers += [nn.Tanh()]
            
        self.model = nn.Sequential(*layers)
        self.model.apply(self.initial_w)
        
    def forward(self, x):
            out = self.model(x)
            return out
        
    #For "tanh",use the 'Xavier' initialization; and for"RELU", use the "he" initialization.
    def initial_w(self, m):
            print(m)
            if type(m) == nn.Linear:
                if self.w_initilization == 'he':
                    nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
                else:
                     nn.init.xavier_uniform_(m.weight)
                        

In [3]:
def train(model, optimizer, criterion, data_loader, epoch):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for batch_idx, (data,target) in enumerate(data_loader):
        data, target = Variable(data), Variable(target)
        output = model(data)
        loss = criterion(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / (batch_idx+1)

In [4]:
def test(model, data_loader, epoch):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        
        for batch_idx, (data,target) in enumerate(data_loader):
                data, target = Variable(data), Variable(target)
                output = model(data)

                total_loss += F.cross_entropy(output, target)

    
    return total_loss/ (batch_idx + 1)    

In [5]:
#Load data and processing

In [6]:
train_df = pd.read_csv("train.csv", header=None)
x_train= np.array(train_df.iloc[:,:-1])
y_train = np.array(train_df.iloc[:,-1])

test_df = pd.read_csv("test.csv", header=None)
x_test = np.array(test_df.iloc[:,:-1])
y_test = np.array(test_df.iloc[:,-1])

In [7]:
y_trainTensor = torch.from_numpy(y_train)
y_testTensor = torch.from_numpy(y_test)
x_trainTensor = torch.from_numpy(np.asarray(x_train).astype(np.float32))
x_testTensor = torch.from_numpy(np.asarray(x_test).astype(np.float32))

In [8]:
train_data = TensorDataset(x_trainTensor, y_trainTensor)
train_loader = DataLoader(train_data,batch_size=12, shuffle=True)
test_data = TensorDataset(x_testTensor, y_testTensor)
test_loader = DataLoader(test_data,batch_size=12, shuffle=False)

In [9]:
depths = [3, 5, 9]
widths = [5, 10, 25, 50, 100]
parameter_type = [['tanh', 'xavier'], ['relu', 'he']]
epoch = 10


In [20]:
depths = [3, 5, 9]
widths = [5, 10, 25, 50, 100]
parameter_type = [['tanh', 'xavier'], ['relu', 'he']]
epoch = 10

for (act, init) in parameter_type:
    for depth in depths:
        for width in widths:
            print('Depth:', depth, ', Widths:', width, ', Activation_function:', act, ', Initialization_type:', init)
            model = NN(4, 2,  width, depth, act, init)
            optimizer = torch.optim.Adam(model.parameters())
            criterion = torch.nn.CrossEntropyLoss()
            train_loss = 0
            test_loss = 0
            for epoch in range(1, epoch + 1):
                train_loss += train(model, optimizer, criterion, train_loader, epoch)
                test_loss += test(model, test_loader, epoch)

            print('train_loss: {:.6f}'.format(train_loss))
            print('test_loss: {:.6f}'.format(test_loss))

Depth: 3 , Widths: 5 , Activation_function: tanh , Initialization_type: xavier
Linear(in_features=4, out_features=5, bias=True)
Tanh()
Linear(in_features=5, out_features=5, bias=True)
Tanh()
Linear(in_features=5, out_features=2, bias=True)
Tanh()
Sequential(
  (0): Linear(in_features=4, out_features=5, bias=True)
  (1): Tanh()
  (2): Linear(in_features=5, out_features=5, bias=True)
  (3): Tanh()
  (4): Linear(in_features=5, out_features=2, bias=True)
  (5): Tanh()
)
train_loss: 3.498996
test_loss: 3.284235
Depth: 3 , Widths: 10 , Activation_function: tanh , Initialization_type: xavier
Linear(in_features=4, out_features=10, bias=True)
Tanh()
Linear(in_features=10, out_features=10, bias=True)
Tanh()
Linear(in_features=10, out_features=2, bias=True)
Tanh()
Sequential(
  (0): Linear(in_features=4, out_features=10, bias=True)
  (1): Tanh()
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): Tanh()
  (4): Linear(in_features=10, out_features=2, bias=True)
  (5): Tanh()
)
train_lo